# Lab | API wrappers - Create your collection of songs & audio features

#### Instructions 


To move forward with the project, you need to create a collection of songs with their audio features - as large as possible! 

These are the songs that we will cluster. And, later, when the user inputs a song, we will find the cluster to which the song belongs and recommend a song from the same cluster.
The more songs you have, the more accurate and diverse recommendations you'll be able to give. Although... you might want to make sure the collected songs are "curated" in a certain way. Try to find playlists of songs that are diverse, but also that meet certain standards.

The process of sending hundreds or thousands of requests can take some time - it's normal if you have to wait a few minutes (or, if you're ambitious, even hours) to get all the data you need.

An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!

In [1]:
import json
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

### Way 1: following example from the internet

see: https://towardsdatascience.com/discovering-your-music-taste-with-python-and-spotify-api-b51b0d2744d

In [3]:
client_id = '65dc8168bce1489ca751fd70199efe57' #insert your client id
client_secret = '26ce94c460e04f3691296058097f8135' # insert your client secret id here

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_1 = 'https://open.spotify.com/playlist/7d3ajghsmx2caVTM4pqIMg?si=a73924480ba844b5' #insert your playlist id
results_1 = sp.playlist(playlist_1)

In [5]:
# create a list of song ids
ids=[]

for item in results_1['tracks']['items']:
        track = item['track']['id']
        ids.append(track)
        
song_meta={'id':[],'album':[], 'name':[], 
           'artist':[],'explicit':[],'popularity':[]}

for song_id in ids:
    # get song's meta data
    meta = sp.track(song_id)
    
    # song id
    song_meta['id'].append(song_id)

    # album name
    album=meta['album']['name']
    song_meta['album']+=[album]

    # song name
    song=meta['name']
    song_meta['name']+=[song]
    
    # artists name
    s = ', '
    artist=s.join([singer_name['name'] for singer_name in meta['artists']])
    song_meta['artist']+=[artist]
    
    # explicit: lyrics could be considered offensive or unsuitable for children
    explicit=meta['explicit']
    song_meta['explicit'].append(explicit)
    
    # song popularity
    popularity=meta['popularity']
    song_meta['popularity'].append(popularity)

song_meta_df=pd.DataFrame.from_dict(song_meta)

# check the song feature
features = sp.audio_features(song_meta['id'])
# change dictionary to dataframe
features_df=pd.DataFrame.from_dict(features)

# convert milliseconds to mins
# duration_ms: The duration of the track in milliseconds.
# 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
features_df['duration_ms']=features_df['duration_ms']/60000

# combine two dataframe
final_df=song_meta_df.merge(features_df)

In [7]:
final_df.shape

(27, 23)

In [10]:
final_df.columns

Index(['id', 'album', 'name', 'artist', 'explicit', 'popularity',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [8]:
final_df

,id,album,name,artist,explicit,popularity,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,0vAio6riryryWWwmkKbBfC,Christmas & Chill,Winter Things,Ariana Grande,False,0,0.507,0.207,0,-10.215,...,0.000000,0.2290,0.774,68.600,audio_features,spotify:track:0vAio6riryryWWwmkKbBfC,https://api.spotify.com/v1/tracks/0vAio6riryry...,https://api.spotify.com/v1/audio-analysis/0vAi...,2.645333,4
1,1OjmlSFuzYflWjSMTCyTJv,Like It's Christmas,Like It's Christmas,Jonas Brothers,False,54,0.725,0.571,1,-6.007,...,0.000000,0.0519,0.610,146.035,audio_features,spotify:track:1OjmlSFuzYflWjSMTCyTJv,https://api.spotify.com/v1/tracks/1OjmlSFuzYfl...,https://api.spotify.com/v1/audio-analysis/1Ojm...,3.348000,4
2,5a1iz510sv2W9Dt1MvFd5R,Christmas (Deluxe Special Edition),It's Beginning to Look a Lot like Christmas,Michael Bublé,False,65,0.339,0.214,4,-11.714,...,0.000007,0.3410,0.363,94.775,audio_features,spotify:track:5a1iz510sv2W9Dt1MvFd5R,https://api.spotify.com/v1/tracks/5a1iz510sv2W...,https://api.spotify.com/v1/audio-analysis/5a1i...,3.444000,3
3,7xapw9Oy21WpfEcib2ErSA,Under The Mistletoe (Deluxe Edition),Mistletoe,Justin Bieber,False,65,0.657,0.560,6,-9.049,...,0.000000,0.0794,0.854,161.993,audio_features,spotify:track:7xapw9Oy21WpfEcib2ErSA,https://api.spotify.com/v1/tracks/7xapw9Oy21Wp...,https://api.spotify.com/v1/audio-analysis/7xap...,3.049117,4
4,0bYg9bo50gSsH3LtXe2SQn,Merry Christmas,All I Want for Christmas Is You,Mariah Carey,False,72,0.336,0.627,7,-7.463,...,0.000000,0.0708,0.350,150.273,audio_features,spotify:track:0bYg9bo50gSsH3LtXe2SQn,https://api.spotify.com/v1/tracks/0bYg9bo50gSs...,https://api.spotify.com/v1/audio-analysis/0bYg...,4.018450,4
5,1ADjWm8QNhgNV8yCNNgQ1T,Santa Tell Me,Santa Tell Me,Ariana Grande,False,0,0.464,0.630,7,-7.337,...,0.000000,0.2950,0.527,132.960,audio_features,spotify:track:1ADjWm8QNhgNV8yCNNgQ1T,https://api.spotify.com/v1/tracks/1ADjWm8QNhgN...,https://api.spotify.com/v1/audio-analysis/1ADj...,3.401550,5
6,1Qi2wh8fFgDV7tl4Sj3f2K,Have Yourself A Merry Little Christmas,Have Yourself A Merry Little Christmas,Sam Smith,False,54,0.438,0.178,5,-9.402,...,0.000000,0.3160,0.344,75.248,audio_features,spotify:track:1Qi2wh8fFgDV7tl4Sj3f2K,https://api.spotify.com/v1/tracks/1Qi2wh8fFgDV...,https://api.spotify.com/v1/audio-analysis/1Qi2...,2.849400,1
7,0RIu23uyX5MTWApqfP549n,Last Christmas,Last Christmas,Ariana Grande,False,0,0.673,0.794,7,-2.758,...,0.000000,0.0934,0.786,102.961,audio_features,spotify:track:0RIu23uyX5MTWApqfP549n,https://api.spotify.com/v1/tracks/0RIu23uyX5MT...,https://api.spotify.com/v1/audio-analysis/0RIu...,3.411333,4
8,0zXjeGMMe7esPTvqJCN5d8,A Very Special Christmas 25th Anniversary,Winter Wonderland,Jason Mraz,False,25,0.609,0.359,5,-6.715,...,0.000000,0.2210,0.645,145.178,audio_features,spotify:track:0zXjeGMMe7esPTvqJCN5d8,https://api.spotify.com/v1/tracks/0zXjeGMMe7es...,https://api.spotify.com/v1/audio-analysis/0zXj...,2.118000,4
9,0mansMnyTsIp9ALspSyLwL,Love Is Everything,Love Is Everything,Ariana Grande,False,0,0.565,0.927,11,-3.473,...,0.000000,0.1680,0.808,180.064,audio_features,spotify:track:0mansMnyTsIp9ALspSyLwL,https://api.spotify.com/v1/tracks/0mansMnyTsIp...,https://api.spotify.com/v1/audio-analysis/0man...,3.562667,4


### Way 2: following the class example

In [24]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="65dc8168bce1489ca751fd70199efe57",
                                                           client_secret="26ce94c460e04f3691296058097f8135"))

In [45]:
results_2 = sp.user_playlist_tracks("spotify", "37i9dQZF1DX0Yxoavh5qJV")
tracks = results_2['items']

for oset in range(100,results_2['total'],100):
    results_2 = sp.user_playlist_tracks("spotify", "37i9dQZF1DX0Yxoavh5qJV", offset=oset)
    tracks += results_2['items']
len(tracks)

123

In [72]:
results_2.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [74]:
results_2['items'][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [90]:
results_2['items'][0]['track']#['album']['name']

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2lolQgalUvZDfp5vvVtTYV'},
    'href': 'https://api.spotify.com/v1/artists/2lolQgalUvZDfp5vvVtTYV',
    'id': '2lolQgalUvZDfp5vvVtTYV',
    'name': 'Tony Bennett',
    'type': 'artist',
    'uri': 'spotify:artist:2lolQgalUvZDfp5vvVtTYV'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/2sw2JVdzZ4lcn2cxh3Nxdc'},
  'href': 'https://api.spotify.com/v1/albums/2sw2JVdzZ4lcn2cxh3Nxdc',
  'id': '2sw2JVdzZ4lcn2cxh3Nxdc',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273835d52eb61628600e466ead9',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e02835d52eb61628600e466ead9',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d00004851835d52eb61628600e466ead9',
    'width': 64}],
  'name': 'Snowfall - The Tony Bennett Christmas Album',
  'release_da

In [82]:
def get_song_from_playlist(playlist=[]):
    
    tracks_from_playlist = playlist
    
    songs = []
    
    for track in tracks_from_playlist:
        songs.append(track['track']['name'])
    
    return list(set(songs))

In [83]:
songs = get_song_from_playlist(tracks)
songs

['Fairytale of New York (feat. Kirsty MacColl)',
 'White Christmas - Spotify Singles - Holiday, Recorded at Air Studios, London',
 'Driving Home for Christmas',
 'Silent Night',
 'Mistletoe',
 'A Holly Jolly Christmas',
 'Merry Christmas Baby',
 'Grown-Up Christmas List',
 'Do You Hear What I Hear? - Remastered 2006',
 'Christmas Time',
 "It's Beginning To Look A Lot Like Christmas",
 'Joy To The World',
 'Step Into Christmas',
 'I Need You Christmas',
 'Happy Xmas (War Is Over)',
 'Man With The Bag',
 'Frosty the Snowman',
 'Frosty The Snowman',
 "It's Christmas",
 'Merry Christmas, Happy Holidays',
 "Rockin' Around The Christmas Tree - Single Version",
 'Snowman',
 'I Saw Mommy Kissing Santa Claus',
 'Oh Holy Night - Recorded at Metropolis Studios, London',
 'Run Rudolph Run',
 'Christmas Without You',
 'The Lighthouse Keeper',
 "It's Not Christmas 'Til You Come Home - Recorded At Spotify Studios NYC",
 'Please Come Home for Christmas - 2013 Remaster',
 "It's the Most Wonderful Time 

In [84]:
def get_song_id_from_playlist(playlist=[]):
    
    tracks_from_playlist = playlist
    
    song_id = []
    
    for track in tracks_from_playlist:
        song_id.append(track['track']['id'])
    
    return list(set(song_id))

In [85]:
song_id = get_song_id_from_playlist(tracks)
song_id

['7uoFMmxln0GPXQ0AcCBXRq',
 '1prYSRBfwPvE3v8jSRZL3Q',
 '6MPpnJDLk37GEIM0iGGITd',
 '5DMItYJluCFc7YtFdP4aXo',
 '2nMZx7QHerfo4Wv37xNUEC',
 '3QIoEi8Enr9uHffwInGIsC',
 '6RS6rcjnWrdfVuu2U2W2dj',
 '7F9oCEKbqjN99cV5HpvYER',
 '2FWKbF1s4s5Z8ZQkyIGNQU',
 '7Hz6LLOVxrojLPIHJJ1S0E',
 '15sxLiiChE5dCW3Y756oas',
 '23lpJkE7jJ4BVQhc3lj0VM',
 '2qoZNAP3JLyIOtbsPtBjvV',
 '1moEucbMhmAK6KVPoNw6ms',
 '5MknXpAUgINT11lxoobaJK',
 '0Ie5uiv54KgCr7P4sYDTHl',
 '2mvabkN1i2gLnGAPUVdwek',
 '32h59T8q2SonUPJ006lyXt',
 '3VTNVsTTu05dmTsVFrmGpK',
 '4i1xD9JHGoRaXcKdmjA4yj',
 '4uQqahi1iUP35xE2qGdKdo',
 '1sFWEpf1aPYN576LS1aa4Y',
 '0YupMLYOYz6lZDbN3kRt7A',
 '0YWUHZPJVg4iujddsJDwhM',
 '5jjsm4Jjq8xjDaue3rW9Go',
 '7inXu0Eaeg02VsM8kHNvzM',
 '4KV9bM7a1KDc7b7OakFZic',
 '1OjmlSFuzYflWjSMTCyTJv',
 '5Q2P43CJra0uRAogjHyJDK',
 '10HDoLPz2bucGBkoNEjDHW',
 '3Xr6kOC4De4Lk41lF9nzTy',
 '2K4XbdyLxi9jhqVQeyUKRK',
 '78pn8k7RogKo2oxl0DyX6d',
 '6yaxdh87KVj82QIYKTP1zt',
 '3nAp4IvdMPPWEH9uuXFFV5',
 '4fzyvSu73BhGvi96p2zwjL',
 '1RMDXedcRno6rDBCbNHDJf',
 

In [57]:
def get_artists_from_playlist(playlist=[]):
    
    tracks_from_playlist = playlist
    
    artists = []
    
    for track in tracks_from_playlist:
        artists_info = track['track']['artists']
        
        for artist_info in artists_info:
            artists.append(artist_info['name'])
    
    return list(set(artists))

In [58]:
artists = get_artists_from_playlist(tracks)
artists

['Coldplay',
 'Ella Fitzgerald',
 'Jennifer Lopez',
 'Mariah Carey',
 'Mel & Kim',
 'Chris Rea',
 'Yoko Ono',
 'ROZES',
 'Fiona Apple',
 'Natalie Taylor',
 'Blake Shelton',
 'Nat King Cole',
 'Paul McCartney',
 'José Feliciano',
 'Chuck Berry',
 'Sia',
 'Tori Kelly',
 'Elvis Presley',
 'Lindsey Stirling',
 'Jordan Smith',
 'Wizzard',
 'Camila Cabello',
 'Perry Como',
 'Liam Payne',
 'Kirsty MacColl',
 'Atlantic Holiday',
 'Jonas Brothers',
 'Tony Bennett',
 'Bing Crosby',
 'Shawn Mendes',
 'Stevie Wonder',
 'Band Aid',
 'Hannah Kerr',
 'The Pogues',
 'The Plastic Ono Band',
 'Gwen Stefani',
 'AJ Mitchell',
 'Josh Groban',
 'Usher',
 "Shakin' Stevens",
 'Troye Sivan',
 'THE ELEVEN',
 'Meghan Trainor',
 'Frank Sinatra',
 'Britney Spears',
 'Bobby Helms',
 'Dixie',
 'Stevie Mackey',
 'The Ronettes',
 'Jessie J',
 'Becky G',
 'Céline Dion',
 'B. Swanson Quartet',
 'Liz Gillies',
 'Earth, Wind & Fire',
 'Mitchell Ayres & His Orchestra',
 'Kelly Clarkson',
 'George Ezra',
 'James TW',
 'Katy

In [92]:
def get_album_from_playlist(playlist=[]):
    
    tracks_from_playlist = playlist
    
    albums = []
    
    for track in tracks_from_playlist:
        albums.append(track['track']['album']['name'])
            
    return list(set(album))

In [96]:
def get_album_from_playlist(playlist=[]):
    
    tracks_from_playlist = playlist
    
    albums = []
    
    for track in tracks_from_playlist:
        album_info = track['track']['album']
        
        for album in album_info:
            albums.append(album_info['name'])
            
    return list(set(album))

In [97]:
albums = get_album_from_playlist(tracks)
albums

['i', 'r', 'u']

--> why does this not work?

In [100]:
df_2 = pd.DataFrame(list(zip(song_id, songs, artists)), columns =['id', 'name', 'artist'])
df_2

,id,name,artist
0,7uoFMmxln0GPXQ0AcCBXRq,Fairytale of New York (feat. Kirsty MacColl),Coldplay
1,1prYSRBfwPvE3v8jSRZL3Q,"White Christmas - Spotify Singles - Holiday, R...",Ella Fitzgerald
2,6MPpnJDLk37GEIM0iGGITd,Driving Home for Christmas,Jennifer Lopez
3,5DMItYJluCFc7YtFdP4aXo,Silent Night,Mariah Carey
4,2nMZx7QHerfo4Wv37xNUEC,Mistletoe,Mel & Kim
...,...,...,...
107,3AgiRs3oxc2dFh0CPUHXJs,Winter Wonderland,Queen
108,1sEC3X76vXgpsgnfaXEMwR,Feliz Navidad,Michael Bublé
109,5ASM6Qjiav2xPe7gRkQMsQ,You Make It Feel Like Christmas (feat. Blake S...,Dean Martin
110,5SyQ5OhqrkJGb35V7RBoL0,Under The Mistletoe,Jamie Cullum


In [103]:
df_2.shape

(112, 3)

In [106]:
# check the song feature
features_1 = sp.audio_features(df_2['id'].iloc[:60])
features_2 = sp.audio_features(df_2['id'].iloc[60:])

In [109]:
len(features_1), len(features_2)

(60, 52)

In [110]:
# change dictionary to dataframe
features_1_df = pd.DataFrame.from_dict(features_1)
features_2_df = pd.DataFrame.from_dict(features_2)

In [115]:
features_df = pd.concat([features_1_df, features_2_df], ignore_index=True)

In [116]:
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.716,0.512,1,-6.257,1,0.0331,0.483,0.000000,0.0928,0.326,104.957,audio_features,7uoFMmxln0GPXQ0AcCBXRq,spotify:track:7uoFMmxln0GPXQ0AcCBXRq,https://api.spotify.com/v1/tracks/7uoFMmxln0GP...,https://api.spotify.com/v1/audio-analysis/7uoF...,165907,3
1,0.504,0.287,1,-11.109,1,0.0346,0.782,0.000002,0.2180,0.707,133.928,audio_features,1prYSRBfwPvE3v8jSRZL3Q,spotify:track:1prYSRBfwPvE3v8jSRZL3Q,https://api.spotify.com/v1/tracks/1prYSRBfwPvE...,https://api.spotify.com/v1/audio-analysis/1prY...,117587,4
2,0.570,0.510,8,-8.148,1,0.0570,0.763,0.000000,0.2850,0.841,76.919,audio_features,6MPpnJDLk37GEIM0iGGITd,spotify:track:6MPpnJDLk37GEIM0iGGITd,https://api.spotify.com/v1/tracks/6MPpnJDLk37G...,https://api.spotify.com/v1/audio-analysis/6MPp...,133773,4
3,0.551,0.325,0,-13.507,1,0.0286,0.810,0.000112,0.1150,0.496,98.932,audio_features,5DMItYJluCFc7YtFdP4aXo,spotify:track:5DMItYJluCFc7YtFdP4aXo,https://api.spotify.com/v1/tracks/5DMItYJluCFc...,https://api.spotify.com/v1/audio-analysis/5DMI...,213160,3
4,0.540,0.734,7,-4.668,1,0.0366,0.365,0.000000,0.1320,0.360,123.877,audio_features,2nMZx7QHerfo4Wv37xNUEC,spotify:track:2nMZx7QHerfo4Wv37xNUEC,https://api.spotify.com/v1/tracks/2nMZx7QHerfo...,https://api.spotify.com/v1/audio-analysis/2nMZ...,216907,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,0.498,0.432,9,-14.386,1,0.0345,0.688,0.000000,0.1360,0.365,135.015,audio_features,3AgiRs3oxc2dFh0CPUHXJs,spotify:track:3AgiRs3oxc2dFh0CPUHXJs,https://api.spotify.com/v1/tracks/3AgiRs3oxc2d...,https://api.spotify.com/v1/audio-analysis/3Agi...,245333,3
108,0.194,0.296,3,-10.429,1,0.0322,0.741,0.000028,0.0862,0.121,91.547,audio_features,1sEC3X76vXgpsgnfaXEMwR,spotify:track:1sEC3X76vXgpsgnfaXEMwR,https://api.spotify.com/v1/tracks/1sEC3X76vXgp...,https://api.spotify.com/v1/audio-analysis/1sEC...,273573,3
109,0.529,0.772,2,-7.013,1,0.0287,0.403,0.000002,0.3160,0.853,91.751,audio_features,5ASM6Qjiav2xPe7gRkQMsQ,spotify:track:5ASM6Qjiav2xPe7gRkQMsQ,https://api.spotify.com/v1/tracks/5ASM6Qjiav2x...,https://api.spotify.com/v1/audio-analysis/5ASM...,181267,4
110,0.650,0.934,5,-4.209,1,0.0474,0.116,0.000000,0.8670,0.737,105.001,audio_features,5SyQ5OhqrkJGb35V7RBoL0,spotify:track:5SyQ5OhqrkJGb35V7RBoL0,https://api.spotify.com/v1/tracks/5SyQ5OhqrkJG...,https://api.spotify.com/v1/audio-analysis/5SyQ...,252200,4


In [117]:
final_df_2 = df_2.merge(features_df, on='id')

In [118]:
final_df_2

,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,7uoFMmxln0GPXQ0AcCBXRq,Fairytale of New York (feat. Kirsty MacColl),Coldplay,0.716,0.512,1,-6.257,1,0.0331,0.483,0.000000,0.0928,0.326,104.957,audio_features,spotify:track:7uoFMmxln0GPXQ0AcCBXRq,https://api.spotify.com/v1/tracks/7uoFMmxln0GP...,https://api.spotify.com/v1/audio-analysis/7uoF...,165907,3
1,1prYSRBfwPvE3v8jSRZL3Q,"White Christmas - Spotify Singles - Holiday, R...",Ella Fitzgerald,0.504,0.287,1,-11.109,1,0.0346,0.782,0.000002,0.2180,0.707,133.928,audio_features,spotify:track:1prYSRBfwPvE3v8jSRZL3Q,https://api.spotify.com/v1/tracks/1prYSRBfwPvE...,https://api.spotify.com/v1/audio-analysis/1prY...,117587,4
2,6MPpnJDLk37GEIM0iGGITd,Driving Home for Christmas,Jennifer Lopez,0.570,0.510,8,-8.148,1,0.0570,0.763,0.000000,0.2850,0.841,76.919,audio_features,spotify:track:6MPpnJDLk37GEIM0iGGITd,https://api.spotify.com/v1/tracks/6MPpnJDLk37G...,https://api.spotify.com/v1/audio-analysis/6MPp...,133773,4
3,5DMItYJluCFc7YtFdP4aXo,Silent Night,Mariah Carey,0.551,0.325,0,-13.507,1,0.0286,0.810,0.000112,0.1150,0.496,98.932,audio_features,spotify:track:5DMItYJluCFc7YtFdP4aXo,https://api.spotify.com/v1/tracks/5DMItYJluCFc...,https://api.spotify.com/v1/audio-analysis/5DMI...,213160,3
4,2nMZx7QHerfo4Wv37xNUEC,Mistletoe,Mel & Kim,0.540,0.734,7,-4.668,1,0.0366,0.365,0.000000,0.1320,0.360,123.877,audio_features,spotify:track:2nMZx7QHerfo4Wv37xNUEC,https://api.spotify.com/v1/tracks/2nMZx7QHerfo...,https://api.spotify.com/v1/audio-analysis/2nMZ...,216907,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,3AgiRs3oxc2dFh0CPUHXJs,Winter Wonderland,Queen,0.498,0.432,9,-14.386,1,0.0345,0.688,0.000000,0.1360,0.365,135.015,audio_features,spotify:track:3AgiRs3oxc2dFh0CPUHXJs,https://api.spotify.com/v1/tracks/3AgiRs3oxc2d...,https://api.spotify.com/v1/audio-analysis/3Agi...,245333,3
108,1sEC3X76vXgpsgnfaXEMwR,Feliz Navidad,Michael Bublé,0.194,0.296,3,-10.429,1,0.0322,0.741,0.000028,0.0862,0.121,91.547,audio_features,spotify:track:1sEC3X76vXgpsgnfaXEMwR,https://api.spotify.com/v1/tracks/1sEC3X76vXgp...,https://api.spotify.com/v1/audio-analysis/1sEC...,273573,3
109,5ASM6Qjiav2xPe7gRkQMsQ,You Make It Feel Like Christmas (feat. Blake S...,Dean Martin,0.529,0.772,2,-7.013,1,0.0287,0.403,0.000002,0.3160,0.853,91.751,audio_features,spotify:track:5ASM6Qjiav2xPe7gRkQMsQ,https://api.spotify.com/v1/tracks/5ASM6Qjiav2x...,https://api.spotify.com/v1/audio-analysis/5ASM...,181267,4
110,5SyQ5OhqrkJGb35V7RBoL0,Under The Mistletoe,Jamie Cullum,0.650,0.934,5,-4.209,1,0.0474,0.116,0.000000,0.8670,0.737,105.001,audio_features,spotify:track:5SyQ5OhqrkJGb35V7RBoL0,https://api.spotify.com/v1/tracks/5SyQ5OhqrkJG...,https://api.spotify.com/v1/audio-analysis/5SyQ...,252200,4


In [119]:
final_df_2.columns, final_df.columns

(Index(['id', 'name', 'artist', 'danceability', 'energy', 'key', 'loudness',
        'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
        'valence', 'tempo', 'type', 'uri', 'track_href', 'analysis_url',
        'duration_ms', 'time_signature'],
       dtype='object'),
 Index(['id', 'album', 'name', 'artist', 'explicit', 'popularity',
        'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'type', 'uri', 'track_href', 'analysis_url', 'duration_ms',
        'time_signature'],
       dtype='object'))

--> final_df_2 is missing 3 columns in comparison: album, explicit, polularity